# Hough Transform implementation

In [58]:
#Performing edge detection -
"""Steps
1.Noise reduction
2.Gradient calculation using sobelx and y
3.Non max suppression
4.edge tracking"""
#Hough Transform & then Line detection and drawing

'Steps\n1.Noise reduction\n2.Gradient calculation using sobelx and y\n3.Non max suppression\n4.edge tracking'

In [66]:
#Noise Reduction using Gaussian filter
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

img = cv2.imread('cube1.jpg', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)
#  Gaussian filter
sigma = 1
sm_img = gaussian_filter(image, sigma=sigma) 

cv2.imshow("image",image)
cv2.imshow("smoothed_image",sm_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [67]:
#Gradient Calculation using Sobel
def sobel_filter(image):
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]])
    grad_x = np.abs(np.convolve(image.ravel(), sobel_x.ravel(), mode='same').reshape(image.shape))
    grad_y = np.abs(np.convolve(image.ravel(), sobel_y.ravel(), mode='same').reshape(image.shape))
    gradient_magnitude = np.sqrt(grad_x ** 2 + grad_y ** 2)
    gradient_direction = np.arctan2(grad_y, grad_x)
    return gradient_magnitude, gradient_direction

In [68]:
gradient_magnitude, gradient_direction = sobel_filter(sm_img)
cv2.imshow("gradient_magnitude",gradient_magnitude)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [69]:
#Non max suppression

height,width = image.shape

def non_maximum_suppression(gradient_magnitude, gradient_direction):
    suppressed_edges = np.zeros_like(gradient_magnitude)
    for i in range(1, height - 1):
        for j in range(1, width - 1):
            angle = gradient_direction[i, j]
            if (0 <= angle < np.pi/8) or (15*np.pi/8 <= angle <= 2*np.pi):
                neighbor1, neighbor2 = gradient_magnitude[i, j+1], gradient_magnitude[i, j-1]
            elif (np.pi/8 <= angle < 3*np.pi/8) or (9*np.pi/8 <= angle < 11*np.pi/8):
                neighbor1, neighbor2 = gradient_magnitude[i+1, j-1], gradient_magnitude[i-1, j+1]
            elif (3*np.pi/8 <= angle < 5*np.pi/8) or (11*np.pi/8 <= angle < 13*np.pi/8):
                neighbor1, neighbor2 = gradient_magnitude[i+1, j], gradient_magnitude[i-1, j]
            else:
                neighbor1, neighbor2 = gradient_magnitude[i-1, j-1], gradient_magnitude[i+1, j+1]
            if gradient_magnitude[i, j] >= neighbor1 and gradient_magnitude[i, j] >= neighbor2:
                suppressed_edges[i, j] = gradient_magnitude[i, j]
    return suppressed_edges



In [70]:
suppressed_edges = non_maximum_suppression(gradient_magnitude, gradient_direction)
cv2.imshow("suppressed_edges",suppressed_edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [71]:
#edge tracking using double thresholding
def hysteresis_thresholding(suppressed_edges, low_threshold, high_threshold):
    strong_edges = (suppressed_edges >= high_threshold)
    weak_edges = (suppressed_edges >= low_threshold) & (suppressed_edges < high_threshold)
    edges_final = np.zeros_like(suppressed_edges, dtype=np.uint8)
    edges_final[strong_edges] = 255
    return edges_final


In [72]:
low_threshold = 15
high_threshold = 60

edges_final = hysteresis_thresholding(suppressed_edges, low_threshold, high_threshold)

cv2.imshow("Edge Tracking", edges_final)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [73]:
#Hough Transform function
def hough_transform(image):
    max_rho = int(np.ceil(np.sqrt(height**2 + width**2)))
    theta_range = np.deg2rad(np.arange(-90, 90))
    rho_range = np.arange(-max_rho, max_rho + 1)
    accumulator = np.zeros((len(rho_range), len(theta_range)), dtype=np.uint64)

    edge_pixels = np.argwhere(image > 0)
    for y, x in edge_pixels:
        for theta_idx, theta in enumerate(theta_range):
            rho = int(x * np.cos(theta) + y * np.sin(theta))
            rho_idx = np.argmin(np.abs(rho_range - rho))
            accumulator[rho_idx, theta_idx] += 1

    return accumulator, rho_range, theta_range

In [74]:
accumulator, rho_range, theta_range = hough_transform(edges_final)

In [103]:
threshold = 100
rho_indices, theta_indices = np.where(accumulator >= threshold)

In [104]:
print(accumulator)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [105]:
print(rho_indices)

[ 460  615  623  696  747  750  882  890  923  930  969  970  975  979
  980 1057 1063 1065 1067 1068 1075 1078 1081 1084 1142 1145 1146 1171
 1172 1176 1180 1181 1184 1185 1187 1188 1189 1190 1267 1271 1273]


In [106]:
print(theta_indices)

[ 33  30  31  29  28  29 168 168 168 168 169 167 168 167 168  90 168 169
 167 168 167 167 166 166  89  90  90 166 166 166 166 165 164 165 164 164
 165 165 165 164 165]


In [107]:
#detecting the lines
detected_lines = []
for rho_idx, theta_idx in zip(rho_indices, theta_indices):
    rho = rho_range[rho_idx]
    theta = theta_range[theta_idx]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b)) 
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    detected_lines.append([(x1, y1), (x2, y2)])


In [108]:
#drawing the lines
image_with_lines = np.stack((image,)*3, axis=-1)
for line in detected_lines:
    cv2.line(image_with_lines, line[0], line[1], (0, 0, 255), 2) 

In [109]:
cv2.imshow("image_with_lines", image_with_lines)
cv2.waitKey(0)
cv2.destroyAllWindows()